In [234]:
import pandas as pd



In [235]:
df1 = pd.read_excel("CO2_emissions/Table 11.01.xlsx", skiprows=8)
df1 = df1[["Month", "Total Energy CO2 Emissions"]]


df2 = pd.read_excel("CO2_emissions/Table 11.02.xlsx", skiprows=8)
df2 = df2[["Month", "Total Energy Residential Sector CO2 Emissions"]]


df3 = pd.read_excel("CO2_emissions/Table 11.03.xlsx", skiprows=8)
df3 = df3[["Month", "Total Energy Commercial Sector CO2 Emissions"]]

df4 = pd.read_excel("CO2_emissions/Table 11.04.xlsx", skiprows=8)
df4 = df4[["Month", "Total Industrial Sector CO2 Emissions"]]


df5 = pd.read_excel("CO2_emissions/Table 11.05.xlsx", skiprows=8)
df5 = df5[["Month", "Total Energy Transportation Sector CO2 Emissions"]]

df6 = pd.read_excel("CO2_emissions/Table 11.06.xlsx", skiprows=8)
df6 = df6[["Month", "Total Energy Electric Power Sector CO2 Emissions"]]


df7 = pd.read_excel("CO2_emissions/Table 11.07.xlsx", skiprows=8)
df7 = df7[["Month", "Biomass CO2 Emissions"]]


In [236]:
dfs = [df1, df2, df3, df4, df5, df6, df7]

df_merged = dfs[0]
for df in dfs[1:]:
    df_merged = df_merged.merge(df, on='Month', how='inner')


In [237]:
cols_to_sum = [
    "Total Energy CO2 Emissions",
    "Total Energy Residential Sector CO2 Emissions",
    "Total Energy Commercial Sector CO2 Emissions",
    "Total Industrial Sector CO2 Emissions",
    "Total Energy Transportation Sector CO2 Emissions",
    "Total Energy Electric Power Sector CO2 Emissions",
    "Biomass CO2 Emissions"
]

df_merged["Total_CO2_Emissions"] = df_merged[cols_to_sum].sum(axis=1)

In [238]:
# df_merged.to_csv("merged_data.csv", index = False)

In [239]:
df_new  = df_merged[["Month","Total_CO2_Emissions"]]

In [240]:
df_new = df_new.drop(index=0).reset_index(drop=True)

In [241]:
df_new.head(5)

,Month,Total_CO2_Emissions
0,1973 January,1031.918
1,1973 February,940.42
2,1973 March,926.209
3,1973 April,841.536
4,1973 May,862.351


In [242]:
df_new[['Year', 'Month_Name']] = df_new['Month'].str.extract(r'(\d{4})\s+(\w+)', expand=True)

month_map = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}
df_new['Month'] = df_new['Month_Name'].map(month_map)
df_new = df_new[["Year", "Month","Total_CO2_Emissions"]]

In [243]:
df_new.head(5)

,Year,Month,Total_CO2_Emissions
0,1973,01,1031.918
1,1973,02,940.42
2,1973,03,926.209
3,1973,04,841.536
4,1973,05,862.351


In [244]:
df_transportation = pd.read_csv("Transportation_petroleum_consumption/MER_T03_07C.csv")
df_transportation = df_transportation[df_transportation["Description"] == "Aviation Gasoline Consumed by the Transportation Sector"]
df_transportation = df_transportation[["YYYYMM","Value"]]


In [245]:
df_transportation.head(5)

,YYYYMM,Value
0,194913,93.129
1,195013,108.266
2,195113,145.071
3,195213,169.391
4,195313,193.732


In [246]:
df_transportation['Year'] = df_transportation['YYYYMM'].astype(str).str[:4]
df_transportation['Month'] = df_transportation['YYYYMM'].astype(str).str[4:]

df_transportation = df_transportation[df_transportation['Month'] != '13'].reset_index(drop=True)
df_transportation = df_transportation[["Year","Month","Value"]]
df_transportation.rename(columns = {"Value" :"Transportarion Petroleum Consumption"}, inplace= True)


In [247]:
df_transportation.head(5)

,Year,Month,Transportarion Petroleum Consumption
0,1973,01,39.516
1,1973,02,44.143
2,1973,03,42.419
3,1973,04,42.567
4,1973,05,50.71


In [248]:
df_new  = df_new.merge(df_transportation, how = "inner", on = ["Year","Month"])

In [249]:
df_new.head(5)

,Year,Month,Total_CO2_Emissions,Transportarion Petroleum Consumption
0,1973,01,1031.918,39.516
1,1973,02,940.42,44.143
2,1973,03,926.209,42.419
3,1973,04,841.536,42.567
4,1973,05,862.351,50.71


In [250]:
df_energy_production = pd.read_csv("Energy_production/MER_T01_02.csv")

In [251]:
df_energy_production['YYYYMM'] = df_energy_production['YYYYMM'].astype(int).astype(str).str.zfill(6)

df_energy_production = df_energy_production[~df_energy_production['YYYYMM'].str[-2:].eq('13')]

df_energy_production['Date'] = pd.to_datetime(df_energy_production['YYYYMM'], format='%Y%m')

df_energy_production['Value'] = pd.to_numeric(df_energy_production['Value'], errors='coerce')
df_energy_production = df_energy_production.pivot_table(
    index='Date',
    columns='Description',
    values='Value'
)
df_energy_production = df_energy_production.sort_index().reset_index()
df_energy_production = df_energy_production[["Date","Total Fossil Fuels Production", "Total Renewable Energy Production"]]

In [252]:
df_energy_production.head(5)

Description,Date,Total Fossil Fuels Production,Total Renewable Energy Production
0,1973-01-01,4.932632,0.219839
1,1973-02-01,4.729582,0.197330
2,1973-03-01,4.946902,0.218686
3,1973-04-01,4.716271,0.209330
4,1973-05-01,4.956995,0.215982


In [253]:
df_energy_production['Year'] = df_energy_production['Date'].dt.year
df_energy_production['Month'] = df_energy_production['Date'].dt.month
df_energy_production['Month'] = df_energy_production['Month'].astype(str).str.zfill(2)
df_energy_production  = df_energy_production[["Year","Month", "Total Fossil Fuels Production", "Total Renewable Energy Production"]]

In [254]:
df_energy_production.head(5)

Description,Year,Month,Total Fossil Fuels Production,Total Renewable Energy Production
0,1973,01,4.932632,0.219839
1,1973,02,4.729582,0.197330
2,1973,03,4.946902,0.218686
3,1973,04,4.716271,0.209330
4,1973,05,4.956995,0.215982


In [255]:

df_new['Year'] = df_new['Year'].astype(int)
df_new['Month'] = df_new['Month'].astype(int)

df_energy_production['Year'] = df_energy_production['Year'].astype(int)
df_energy_production['Month'] = df_energy_production['Month'].astype(int)


df_merged = df_new.merge(df_energy_production, how="inner", on=["Year", "Month"])


In [256]:
df_merged.head(12)

,Year,Month,Total_CO2_Emissions,Transportarion Petroleum Consumption,Total Fossil Fuels Production,Total Renewable Energy Production
0,1973,1,1031.918,39.516,4.932632,0.219839
1,1973,2,940.42,44.143,4.729582,0.197330
2,1973,3,926.209,42.419,4.946902,0.218686
3,1973,4,841.536,42.567,4.716271,0.209330
4,1973,5,862.351,50.71,4.956995,0.215982
5,1973,6,841.438,44.967,4.716096,0.208249
6,1973,7,872.652,38.774,4.748276,0.207800
7,1973,8,908.028,61.71,5.041997,0.203432
8,1973,9,845.914,44.133,4.731504,0.185300
9,1973,10,892.251,50.806,4.998321,0.193514
